# Projeto de sistema de recomendação de filmes
## Passo 1) Lendo o dataset

O dataset utilizado é o MovieLens, cujo detalhamento pode ser encontrado no link http://files.grouplens.org/datasets/movielens/ml-20m-README.html

Os arquivos são disponibilizados no formato '.csv'. Assim, faremos a leitura dos arquivos com a biblioteca Pandas. 

Deve, neste ponto, também ser separado um conjunto utilizado para a recomendação e outro para a avaliação do sistema, com o auxílio da biblioteca Scikit-learn. O método de divisão estratificada será utilizado para manter a proporção de avaliações para todos os usuários em ambos os conjuntos. Eles serão divididos à proporção de 70% treino e 30% teste, ficando com cerca de 14mi de avaliações no conjunto principal e 6mi de avaliações no conjunto de teste.

In [1]:
# Importando os pacotes necessários
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
#import matplotlib.pyplot as plt
import re

#%matplotlib inline

In [2]:
# Lendo os arquivos
ratings = pd.read_csv('ml-20m/ratings.csv')
movies = pd.read_csv('ml-20m/movies.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

# Mesclando os dados dos filmes com as avaliações
ratings_movies_all = pd.merge(ratings, movies, on='movieId').drop('timestamp', axis=1)

# Separando os índices para os conjuntos de treino e teste
split = StratifiedShuffleSplit(n_splits=1,test_size=0.3)
indices_train, indices_test = next(split.split(np.zeros(len(ratings_movies_all['userId'])), ratings_movies_all['userId']))

# Definindo o conjunto de treino, ou principal
ratings_movies = ratings_movies_all.iloc[indices_train]
ratings_movies

,userId,movieId,rating,title,genres
16928982,130406,1099,5.0,"Christmas Carol, A (1938)",Children|Drama|Fantasy
7884701,50992,592,5.0,Batman (1989),Action|Crime|Thriller
8607917,65431,3247,4.0,Sister Act (1992),Comedy|Crime
5507342,65157,588,1.5,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
16260681,1551,85774,3.5,Senna (2010),Documentary
19327487,126135,1837,5.0,"Odd Couple II, The (1998)",Comedy
13367582,112653,542,2.0,Son in Law (1993),Comedy|Drama|Romance
14550592,130432,43,4.0,Restoration (1995),Drama
1644473,932,2542,4.5,"Lock, Stock & Two Smoking Barrels (1998)",Comedy|Crime|Thriller
15536732,100254,1693,3.0,Amistad (1997),Drama|Mystery


In [3]:
# Definindo o conjunto de testes
ratings_movies_test = ratings_movies_all.iloc[indices_test]
ratings_movies_test

,userId,movieId,rating,title,genres
12944420,138012,5464,3.5,Road to Perdition (2002),Crime|Drama
13292339,134728,382,2.5,Wolf (1994),Drama|Horror|Romance|Thriller
4093766,38848,1917,1.0,Armageddon (1998),Action|Romance|Sci-Fi|Thriller
10952710,86398,1371,3.0,Star Trek: The Motion Picture (1979),Adventure|Sci-Fi
3848779,46577,1276,2.0,Cool Hand Luke (1967),Drama
18231430,52457,6078,2.5,Firefox (1982),Action|Sci-Fi|Thriller
15483490,107186,361,4.0,It Could Happen to You (1994),Comedy|Drama|Romance
19741999,13343,33539,4.5,Deep Blue (2003),Documentary
15216819,119449,2522,2.0,Airport '77 (1977),Drama
11032410,74704,2402,4.0,Rambo: First Blood Part II (1985),Action|Adventure|Thriller


## Passo 2) Como explorar os dados?

Neste ponto a intenção é conhecer melhor os dados, explorando através de questões. Primeiro respondemos questões simples e mais genéricas, como os 10 filmes com mais avaliações 5 estrelas. Depois partimos para questões mais complexas relacionadas a preferência geral.

Deste ponto em diante serão utilizados somente os dados do conjunto de treino para que não haja interferência na avaliação do sistema de recomendação.

In [4]:
# Filtrar os 10 filmes com maior NÚMERO de avaliações 5 estrelas, listando-os pelo título:
top_5star_movies = ratings_movies[ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_movies

Shawshank Redemption, The (1994)             22311
Pulp Fiction (1994)                          19474
Silence of the Lambs, The (1991)             15698
Schindler's List (1993)                      15650
Star Wars: Episode IV - A New Hope (1977)    15465
Forrest Gump (1994)                          14905
Godfather, The (1972)                        14068
Usual Suspects, The (1995)                   13845
Matrix, The (1999)                           13006
Braveheart (1995)                            12911
Name: title, dtype: int64

In [5]:
# Os 20 filmes com maior MÉDIA de estrelas, listando-os pelo título:
top_meanstar = ratings_movies.groupby('title').agg({'rating': [np.size, np.mean]})
top_meanstar.sort_values([('rating', 'mean')], ascending=False).head(20)

rating     
                                                     size mean
title                                                         
Alcina (2000)                                         1.0  5.0
Shaolin Temple 2: Kids from Shaolin (Shao Lin x...    1.0  5.0
God’s Wedding (As Bodas de Deus) (1999)               1.0  5.0
Moth, The (Cma) (1980)                                1.0  5.0
Pirates of the Great Salt Lake (2006)                 1.0  5.0
Blue Swallow (Cheong yeon) (2005)                     1.0  5.0
Death of a Nation - The Timor Conspiracy (1994)       1.0  5.0
Schmatta: Rags to Riches to Rags (2009)               1.0  5.0
Codes of Gender, The (2010)                           1.0  5.0
The Green (2011)                                      1.0  5.0
Rent-a-Cat (2012)                                     1.0  5.0
Aashiqui 2 (2013)                                     1.0  5.0
Chhoti Si Baat (1975)                                 1.0  5.0
Bo Burnham: Words, Words, Words (2010)                1.0  5.0
Small Roads (2011)                                    1.0  5.0
The Private Life of a Cat (1944)                      1.0  5.0
New Rulers of the World, The (2001)                   1.0  5.0
People of the Wind (1976)                             1.0  5.0
Broken Sky (El cielo dividido) (2006)                 1.0  5.0
Mutantes (2009)                                       1.0  5.0

##### Diferença entre usar o número de avaliações 5 estrelas e a média de estrelas por filme:

Ao utilizar a número de avaliações 5 estrelas nós selecionamos os títulos mais populares e bem avaliados, entretanto, podem ficar subamostrados os bons títulos mas com poucas avaliações. Neste ponto supre tal necessidade o uso da média de estrelas por título, mas este último método também oferece a desvantagem de selecionar títulos que não são populares e também títulos que tiveram pouquissimas avaliações, contudo positivas.

### Questões genéricas relacionadas aos gêneros

* Quais os filmes com mais avaliações 5 estrelas dentro de cada gênero?

* Podemos listar filmes com mais de um gênero descritos?

In [6]:
# Um exemplo para filtrar os filmes por gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama = ratings_movies[ratings_movies['genres'].str.contains('Drama')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Shawshank Redemption, The (1994)    22311
Pulp Fiction (1994)                 19474
Schindler's List (1993)             15650
Forrest Gump (1994)                 14905
Godfather, The (1972)               14068
Braveheart (1995)                   12911
American Beauty (1999)              11059
Fargo (1996)                        10769
Fight Club (1999)                   10276
Godfather: Part II, The (1974)       8234
Name: title, dtype: int64

In [7]:
# Um exemplo para filtrar os filmes por mais de um gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama_romance = ratings_movies[ratings_movies['genres'].str.contains('Drama' and 'Romance')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama_romance

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Forrest Gump (1994)                                     14905
Princess Bride, The (1987)                               8427
Casablanca (1942)                                        7275
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)     5562
Beauty and the Beast (1991)                              5056
Good Will Hunting (1997)                                 4740
Life Is Beautiful (La Vita è bella) (1997)               4641
Sense and Sensibility (1995)                             4551
Groundhog Day (1993)                                     4477
Shakespeare in Love (1998)                               4301
Name: title, dtype: int64

### Questões relacionadas a preferência por gênero

Usuários avaliam apenas um gênero, ou mais gêneros? Eles gostam desses gêneros? O quão importante é o gênero na escolha e avaliação do filme?

Para responder a tal pergunta nós seguiremos as seguintes etapas: 

 1. Determinar o número de avaliações por gênero e usuário
 
 2. Determinar a participação de cada gênero no número de avaliações de cada usuário
 
 3. Encontrar quais os gêneros que compõe a maior parte das avaliações do usuário (determinar um threshold)
 
 4. Determinar a avaliação média do usuário para cada gênero identificado
 
 5. Metrificar a importância do gênero na escolha e na avaliação do filme através dos dados anteriores, por exemplo a média de número de gêneros considerados importantes e a médias das avaliações em tais gêneros.



In [8]:
# 1. Determinando o número de avaliações por gênero para cada usuário

# 1a) definindo todos os gêneros que existem:
genre_labels = set()
for s in ratings_movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# 1b) desmembrando todos os gêneros em colunas separadas
genres_df = pd.DataFrame(dict((genre, ratings_movies['genres'].str.contains(genre, re.IGNORECASE))
                             for genre in genre_labels))
ratings_movies_expand = genres_df.join(ratings_movies)

# 1c) contar quantas vezes os gêneros são verdadeiros por usuário
n_gen_user = ratings_movies_expand.groupby('userId').agg({genre:[np.sum] for genre in genre_labels})

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


In [9]:
# 2. Determinando a participação percentual de cada gênero no número de avaliações de cada usuário
n_gen_user['total'] = n_gen_user.apply(sum, axis=1)
tax_gen_user = n_gen_user.div(n_gen_user['total'], axis=0).mul(100)

In [10]:
# 3. Encontrando qual(is) gênero(s) que compõe(m) a maior parte das avaliações do usuário
tax_pref = tax_gen_user.drop('total', axis=1)
preference = pd.DataFrame()
preference['genre_max'] = tax_pref.apply(np.argmax, axis=1)
preference['value_max'] = tax_pref.apply(np.amax, axis=1)

preference.head()

C:\Users\mariv\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,genre_max,value_max
userId,,
1,"(Adventure, sum)",14.166667
2,"(Sci-Fi, sum)",14.705882
3,"(Sci-Fi, sum)",18.208092
4,"(Thriller, sum)",14.754098
5,"(Adventure, sum)",14.062500


In [11]:
# 4. Determinar a avaliação média do usuário para cada gênero identificado

In [12]:
# 5. Metrificar a importância do gênero na escolha e na avaliação do filme

## Passo 3) Sistema de recomendação

O sistema de recomendação será baseado primeiramente no(s) gênero(s) que o usuário mais avalia, e depois no número de avaliações 5 estrelas dadas pelo universo de usuários.

O sistema seguirá os seguintes passos:

1. Filtrar lista de filmes que possuem o(s) gênero(s) de preferência do usuário e avaliação maior que 4.5

2. Excluir os filmes que o usuário já assitiu

3. Fazer um ranking desta lista de filmes de acordo com o número de avaliações 5 estrelas

4. Sugerir os filmes no topo do ranking (1 ou mais)

In [13]:
# exemplo com um usuário:
user = 3
pref_genre = 'Sci-Fi'

# 1. Filtrar lista de filmes que possuem o gênero definido e avaliação 5 estrelas
suggestion = ratings_movies[ratings_movies['genres'].str.contains(pref_genre)][ratings_movies['rating'] > 4.5]

# 2. Excluir o que o usuário já assistiu
viewed = ratings_movies[ratings_movies['userId'] == user]['movieId']

for movie in viewed:
    suggestion = suggestion[suggestion['movieId'] != movie]

# 3 e 4. Rankear pelo maior número de avaliações 5 estrelas e sugerir os melhores
n_suggestions = 5
final_suggestion = suggestion['title'].value_counts()[:n_suggestions]
final_suggestion

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Star Wars: Episode V - The Empire Strikes Back (1980)    11822
Star Wars: Episode VI - Return of the Jedi (1983)         9219
Jurassic Park (1993)                                      7204
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                 6743
Alien (1979)                                              6114
Name: title, dtype: int64

## Passo 3) Avaliação

Para fazer a avaliação será utilizado o conjunto de teste definido no início. Este conjunto contém avaliações de todos os usuários em um percentual de 30% em relação ao total de avaliações. 
A avaliação seguirá os seguintes passos.

1. Determinar se os filmes sugeridos constam no conjunto de teste para cada usuário.

2. Se houver filmes sugeridos no conjunto de testes, somar e fazer a média do número de estrelas dadas a estes filmes pelo usuário.

Estes números obtidos para cada usuário para cada usuário poderá ser manipulado para se tornar um índice único para avaliação do sistema ou avaliado para todos os usuários correlacionando-os com outros dados como o gênero de preferência, ou ainda ao índice de importância do gênero definido na exploração inicial dos dados.


In [14]:
# 1. Determinar se os filmes sugeridos constam no conjunto de teste para cada usuário

# 1a) Listar as avaliações do conjunto de testes do usuário.
test_viewed = ratings_movies_test[ratings_movies_test['userId'] == user] 

# 1b) Verificar o que foi sugerido pelo sistema de recomendação 
evaluate = np.zeros(len(test_viewed), dtype=bool)
i = 0
for movie in test_viewed['title']:
    evaluate[i] = movie in final_suggestion
    i = i+1
test_viewed['saw'] = evaluate
test_viewed.head()

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,userId,movieId,rating,title,genres,saw
4326697,3,2428,4.0,"Faculty, The (1998)",Horror|Sci-Fi,False
3333734,3,718,3.0,"Visitors, The (Visiteurs, Les) (1993)",Comedy|Fantasy|Sci-Fi,False
896360,3,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama,False
4355597,3,2530,4.0,Beneath the Planet of the Apes (1970),Action|Sci-Fi,False
4450127,3,2642,3.0,Superman III (1983),Action|Adventure|Sci-Fi,False


In [15]:
# 2. Se houver filmes sugeridos no conjunto de testes, somar o número de estrelas dadas a estes filmes pelo usuário.
sum_rating_sugg = test_viewed[test_viewed['saw'] == True]['rating'].sum()
mean_rating_sugg = test_viewed[test_viewed['saw'] == True]['rating'].mean()
print('Soma das estrelas dadas aos filmes sugeridos: ', sum_rating_sugg, 'de 25.0')
print('Média de estrelas dadas aos filmes sugeridos: ', mean_rating_sugg)

Soma das estrelas dadas aos filmes sugeridos:  24.0 de 25.0
Média de estrelas dadas aos filmes sugeridos:  4.8
